In [1]:
import time
import os
import re
import pandas as pd
import numpy as np
from abc import ABC
from tqdm.notebook import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from typing import Literal, Tuple, Iterable
import platform
from dotenv import load_dotenv
from urllib.parse import urlparse

load_dotenv()

True

In [2]:
SCREEN_WIDTH = 1470
SCREEN_HEIGHT = 956
HEADLESS = False
MAX_TIME_TO_WAIT = 20

In [3]:
class BaseScraper(ABC):
    def __init__(self, options: Options, credentials: dict, max_time_to_wait: float) -> None:
        ...

    def __setup_driver(self, options: Options) -> None:
        ...

    def login(self) -> None:
        ...    

    def get_grades_df(self) -> pd.DataFrame:
        ...

    def shutdown(self) -> None:
        ...

In [4]:
class SigaaScraper(BaseScraper):
    def __init__(self, options: Options, credentials: dict[Literal['username', 'password'], str], max_time_to_wait: float, sigaa_base_url: str) -> None:
        self.__setup_driver(options)
        self.__credentials = credentials
        self.__max_time_to_wait = max_time_to_wait
        self.__sigaa_base_url = sigaa_base_url  # Exemplo: http://sigaa.ufpe.br/sigaa/


    def __setup_driver(self, options):
        self.__driver = webdriver.Chrome(options=options)


    def login(self) -> None:
        # Acessa a página do UFABC Next
        self.__driver.get(self.__sigaa_base_url)

        # Localiza o campo de usuário e senha
        input_username = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
            EC.presence_of_element_located((By.NAME, 'user.login'))
        )
        input_username.send_keys(self.__credentials['username'])
        input_pswd = self.__driver.find_element(by=By.NAME, value='user.senha')  # Não necessário usar o WebDriverWait -> Se já encontrou um, o outro tem que estar lá também 
        input_pswd.send_keys(self.__credentials['password'])
        submit_button = self.__driver.find_element(By.XPATH, '//input[@type="submit" and @value="Entrar"]')
        submit_button.click()


    def __go_to_grades_page(self) -> None:
        # Seta a página base do "Portal do Discente" -> URL base seguida do path abaixo
        portal_url = urlparse(self.__sigaa_base_url)._replace(path='sigaa/portais/discente/discente.jsf').geturl()
        # Vai para a página "Portal do Discente" (passo a mais)
        self.__driver.get(portal_url)

        # Clica na aba "Ensino" e depois "Consultar Minhas Notas"
        dropdown = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
            EC.element_to_be_clickable((By.XPATH, '//span[@class="ThemeOfficeMainFolderText" and text()="Ensino"]'))
        )
        dropdown.click()
        get_grades_opt = self.__driver.find_element(by=By.XPATH, value='//td[@class="ThemeOfficeMenuItemText" and text()="Consultar Minhas Notas"]')
        get_grades_opt.click()


    def get_grades_df(self) -> pd.DataFrame:
        # Puxa os dados da tabela
        self.__go_to_grades_page()

        # Puxa os dados da tabela
        tables_elements = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
            EC.visibility_of_all_elements_located((By.XPATH, '//table[@class="tabelaRelatorio"]'))
        )

        df_list = []
        for table in tables_elements:
            # Agora vamos extrair as linhas da tabela
            year = table.find_element(By.TAG_NAME, 'caption').text
            rows = table.find_elements(By.TAG_NAME, 'tr')

            # Extraindo os cabeçalhos (primeira linha da tabela)
            headers = [header.text for header in rows[0].find_elements(By.TAG_NAME, 'th')]

            # Extraindo as linhas de dados
            data = []
            for row in rows[1:]:  # pulando o cabeçalho
                columns = row.find_elements(By.TAG_NAME, 'td')
                row_data = [col.text for col in columns]
                data.append(row_data)

            # Convertendo os dados extraídos para um DataFrame do Pandas
            df_tmp = pd.DataFrame(data, columns=headers)
            df_tmp['Ano'] = year
            
            df_list.append(df_tmp)
        
        df = pd.concat(df_list)
        df = df[['Ano', 'Código', 'Disciplina', 'Resultado', 'Situação']]

        return df


    def shutdown(self):
        self.__driver.quit()
    

In [5]:
options = Options()
options.add_argument('--disable-gpu')  # Desabilita a aceleração de GPU (opcional)
options.add_argument('--start-maximized')  # Deixa em tela cheia
options.add_argument('--window-position=0,-1800')  # Para mover para o monitor de cima
# options.add_argument('--headless')  # Ativa o modo headless

credentials = {
    'username': os.environ['CIN-USERNAME'],
    'password': os.environ['CIN-PASSWORD']
}

ufpe_scraper = SigaaScraper(options=options, credentials=credentials, max_time_to_wait=10, sigaa_base_url='https://sigaa.ufpe.br/sigaa/')
ufpe_scraper.login()
df_ufpe = ufpe_scraper.get_grades_df()
ufpe_scraper.shutdown()

df_ufpe

,Ano,Código,Disciplina,Resultado,Situação
0,2024.1,CIN0068,APLICAÇÕES DE APRENDIZAGEM DE MÁQUINA,"9,9",APROVADO
1,2024.1,CIN0066,APRENDIZAGEM DE MÁQUINA I,"10,0",APROVADO
2,2024.1,CIN0067,APRENDIZAGEM DE MÁQUINA II,"10,0",APROVADO
3,2024.1,CIN0069,ARQUITETURA BIG DATA E ANALYTICS,"10,0",APROVADO
4,2024.1,CIN0061,ARQUITETURA DE SOFTWARE,"10,0",APROVADO
5,2024.1,CIN0065,ESTATÍSTICA AVANÇADA,"9,5",APROVADO
6,2024.1,CIN0071,FAMILIARIZAÇÃO AERONÁUTICA,"10,0",APROVADO
7,2024.1,CIN0064,INTRODUÇÃO A CIÊNCIA DE DADOS,"9,5",APROVADO
8,2024.1,CIN0070,PROJETO DE CIÊNCIA DE DADOS,"10,0",APROVADO
9,2024.1,CIN0062,QUALIDADE DE SOFTWARE,"10,0",APROVADO


In [6]:
options = Options()
options.add_argument('--disable-gpu')  # Desabilita a aceleração de GPU (opcional)
options.add_argument('--window-position=0,-1800')  # Para mover para o monitor de cima
options.add_argument('--start-maximized')  # Deixa em tela cheia
# options.add_argument('--headless')  # Ativa o modo headless

credentials = {
    'username': os.environ['UFABC-USERNAME'],
    'password': os.environ['UFABC-PASSWORD']
}

ufabc_scraper = SigaaScraper(options=options, credentials=credentials, max_time_to_wait=10, sigaa_base_url='http://sig.ufabc.edu.br/sigaa')
ufabc_scraper.login()
df_ufabc = ufabc_scraper.get_grades_df()
df_ufabc = df_ufabc[df_ufabc['Situação'] != 'CANCELADO'].sort_values('Ano', ascending=True)
ufabc_scraper.shutdown()

df_ufabc

,Ano,Código,Disciplina,Resultado,Situação
5,2017.2,BIL0304-15,EVOLUÇÃO E DIVERSIFICAÇÃO DA VIDA NA TERRA,C,APROVADO
0,2017.2,BCS0001-15,BASE EXPERIMENTAL DAS CIÊNCIAS NATURAIS,A,APROVADO
1,2017.2,BIS0005-15,BASES COMPUTACIONAIS DA CIÊNCIA,A,APROVADO
4,2017.2,BIK0102-15,ESTRUTURA DA MATÉRIA,A,APROVADO
3,2017.2,BIS0003-15,BASES MATEMÁTICAS,C,APROVADO
...,...,...,...,...,...
1,2023.2,ESTA017-17,LABORATÓRIO DE MÁQUINAS ELÉTRICAS,A,APROVADO
0,2023.2,ESTA011-17,AUTOMAÇÃO DE SISTEMAS INDUSTRIAIS,A,APROVADO
2,2023.3,ESTA904-17,TRABALHO DE GRADUAÇÃO III EM ENGENHARIA DE INS...,A,APROVADO
1,2023.3,ESTA022-17,TEORIA DE ACIONAMENTOS ELÉTRICOS,A,APROVADO


In [ ]:
class UFABCNextGradeScraper(BaseScraper):
    def __init__(self, options: Options, credentials: dict[Literal['e-mail', 'ra'], str], max_time_to_wait: float) -> None:
        self.__setup_driver(options)
        self.__credentials = credentials
        self.__max_time_to_wait = max_time_to_wait


    @staticmethod
    def __standardize_name(name: str) -> Tuple[str, str]:
        name_std = name.lower().strip()
        name_splitted_list = re.split(r'[áéíóúàãõâêôüàç,.:/|]', name_std)
        name_splitted = sorted(name_splitted_list, key=lambda s: -len(s))[0]  # Pega a maior parte

        return name_std, name_splitted
    

    @staticmethod
    def print_grades_dict(grades_dict: dict) -> None:
        for grade, percentual in grades_dict.items():
            print(f'{grade}: {percentual}%')


    def __setup_driver(self, options: Options) -> None:
        self.__driver = webdriver.Chrome(options=options)


    def login(self) -> None:
        # Acessa a página do UFABC Next
        self.__driver.get('https://ufabcnext.com')

        # Localiza botão do Facebook e clica
        facebook_button = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'facebook-button'))
        )
        facebook_button.click()

        # Escreve o e-mail , RA e depois submete para fazer o login
        input_email = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
            EC.presence_of_element_located((By.ID, 'input-0'))
        )
        input_email.send_keys(self.__credentials['e-mail'])
        input_ra = self.__driver.find_element(by=By.ID, value='input-2')  # Não necessário usar o WebDriverWait -> Se já encontrou um, o outro tem que estar lá também 
        input_ra.send_keys(self.__credentials['ra'])
        submit_button = self.__driver.find_element(by=By.CSS_SELECTOR, value='button[type="submit"]')
        submit_button.click()


    def get_subject_grades_prop(self, subject_name: str) -> dict:
        # Padroniza os nomes para conseguir fazer a busca no Next
        subject_name_std, subject_name_splitted = self.__standardize_name(subject_name)

        # Procura por uma matéria em específico
        subjects_input = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
            EC.presence_of_element_located((By.ID, 'input-20'))
        )

        if platform.system() == 'Darwin':  # Mac 
            subjects_input.send_keys(Keys.COMMAND + 'a')  # Seleciona tudo (COMMAND pois é Mac)
        else:
            subjects_input.send_keys(Keys.CONTROL + 'a')

        subjects_input.send_keys(Keys.DELETE)  # Apaga o que acabou de selecionar
        subjects_input.send_keys(subject_name_splitted)
        option = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
            # EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'v-list-item')][contains(., '{subject_name_std}')]"))  # Se o texto conter aquilo
            EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'v-list-item')][text()=' {subject_name_std}']"))  # Se o texto for igual! Tem sempre um espacinho na frente. Pode ser perigoso se houver uma pequena diferença!
        )
        option.click()

        # Puxa os valores e imprime na tela
        grades_percentage = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
            EC.visibility_of_all_elements_located((By.CLASS_NAME, 'highcharts-text-outline'))
        )
        grades_dict = {gp[0]: gp[1].split('%')[0].strip() for gp in [gp_.text.split(':') for gp_ in grades_percentage]}

        return grades_dict
    

    def get_grades_df(self, subjects_list: Iterable) -> pd.DataFrame:
        subjects_dict = {}
        error_subjects_list = []
        
        for subject in tqdm(subjects_list):
            try:
                grades_dict = self.get_subject_grades_prop(subject)    
                subjects_dict[subject] = grades_dict

            except Exception:
                error_subjects_list.append(subject)

            finally:
                time.sleep(2)  # Para evitar de tomar cooldown

        df = pd.DataFrame.from_dict(subjects_dict).T.reset_index().rename(columns={'index': 'Disciplina'}).fillna(0)
        df['F'] = df['F'].astype(float) + df['O'].astype(float)
        df = df.drop(columns=['O'])
        df[['A', 'B', 'C', 'D', 'F']] = df[['A', 'B', 'C', 'D', 'F']].astype(float)
        df['Nota provável'] = df[['A', 'B', 'C', 'D', 'F']].apply(lambda s: (s['A']*4 + s['B']*3 + s['C']*2 + s['D']*1)/s[['A', 'B', 'C', 'D', 'F']].sum(), axis=1)
        df['Conceito provável'] = df['Nota provável'].apply(lambda s: 'F' if 0.0 <= s < 0.8 
                                                                            else 'D' if 0.8 <= s < 1.6
                                                                            else 'C' if 1.6 <= s < 2.4
                                                                            else 'B' if 2.4 <= s < 3.2
                                                                            else 'A')
        df['Conceito moda'] = df[['A', 'B', 'C', 'D', 'F']].apply(lambda s: np.argmax(s), axis=1).map({0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'F'})
        
        for error_subject in error_subjects_list:
            df = pd.concat([df, pd.DataFrame({'Disciplina': [error_subject]})])
        
        df = df.reset_index(drop=True)

        return df
    

    def shutdown(self) -> None:
        self.__driver.quit()


In [8]:
options = Options()
options.add_argument('--disable-gpu')  # Desabilita a aceleração de GPU (opcional)
options.add_argument('--start-maximized')  # Deixa em tela cheia
options.add_argument('--window-position=0,-1800')  # Para mover para o monitor de cima
# options.add_argument('--headless')  # Ativa o modo headless

credentials = {
    'e-mail': os.environ['EMAIL'],
    'ra': os.environ['RA']
}

max_time_to_wait = 10

subjects = df_ufabc['Disciplina'].values

scraper = UFABCNextGradeScraper(options=options, credentials=credentials, max_time_to_wait=max_time_to_wait)
scraper.login()
df_other = scraper.get_grades_df(subjects)
scraper.shutdown()

df_other

  0%|          | 0/78 [00:00<?, ?it/s]

,Disciplina,A,B,C,D,F,Nota provável,Conceito provável,Conceito moda
0,EVOLUÇÃO E DIVERSIFICAÇÃO DA VIDA NA TERRA,31.0,43.1,17.4,4.8,3.7,2.929000,B,B
1,BASE EXPERIMENTAL DAS CIÊNCIAS NATURAIS,69.6,25.1,3.8,0.4,1.1,3.617000,A,A
2,BASES COMPUTACIONAIS DA CIÊNCIA,39.2,26.9,18.3,7.3,8.4,2.811189,B,A
3,ESTRUTURA DA MATÉRIA,17.5,29.5,31.4,10.4,11.3,2.314685,C,C
4,BASES MATEMÁTICAS,8.8,15.4,25.9,15.5,34.5,1.485514,D,F
...,...,...,...,...,...,...,...,...,...
73,TEORIA DE ACIONAMENTOS ELÉTRICOS,25.4,29.4,22.7,10.2,12.4,2.451548,B,B
74,SISTEMAS DE CONTROLE II,12.1,21.4,24.3,15.5,26.7,1.767000,C,F
0,FENÔMENOS MECÂNICOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,FOTÔNICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
